# TP1 : Modélisation Sémantique des données


 ***Gaoussou SANOU et Konstantin TODOROV***

In [1]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF, RDFS, OWL, XSD

namespace = "http://mygraph.com/"
dbpedia = "http://dbpedia.org/ontology/"
foaf = "http://xmlns.com/foaf/0.1/"
kzk = "http://www.kanzaki.com/ns/music#"
schema = "http://schema.org/"
%reload_ext lab_black

ModuleNotFoundError: No module named 'rdflib'

Wolfgang Amadeus Mozart, né à Salzbourg le 27 janvier 1756, mort à Vienne, la capitale autrichienne, le 5 décembre 1791, est un compositeur.
Il est fils de Léopold Mozart et était marié à Constance Weber. Parmi ses œuvres les plus célèbres, on trouve l'opéra “La flûte enchantée” et la 41 Symphonie en ut majeur, dite “Jupiter”.
Cette symphonie est composée de 4 parties : 1. Allegro Vivace, 2. Andante Cantabile, 3. Menuetto, 4. Molto Allegro. Elle a été enregistrée par l'orchestre symphonique de Londres sous la direction de Claudio Abbado en 1980.

Extraire manuellement les connaissances exprimées dans ce texte : penser aux entités (personnes, œuvres, lieux, etc.), aux relations entre ces entités (« fils », « compositeur », « capitale », etc.) et à leur caractéristiques (l’œuvre « Jupiter »; est dans le genre “symphonie”, etc.). Présenter cela sous la forme d'une table. https://lov.linkeddata.es/

### Concepts

In [ ]:
location = URIRef(f"https://w3id.org/dpv#Location")
artwork = URIRef(f"{dbpedia}Artwork")
country = URIRef(f"{dbpedia}Country")
town = URIRef(f"{dbpedia}Town")
hometown = URIRef(f"{dbpedia}hometown")
compositeur = URIRef(f"{kzk}music#Composer")

### Relations

In [ ]:
isDead = URIRef(f"{namespace}isDead")
isborn = URIRef(f"{kzk}whois#born")
hascapital = URIRef(f"{dbpedia}capital")
birthplace = URIRef(f"http://schema.org/birthPlace")
deadplace = URIRef(f"{schema}deathPlace")
hasspouse = URIRef(f"{dbpedia}spouse")
sonOf = URIRef(f"http://sparql.cwrc.ca/ontologies/cwrc#sonOf")
hasPart = URIRef(f"{schema}hasPart")
direct_by = URIRef(f"http://vocab.getty.edu/ontology#ulan2575_directed_by")
birthdate = URIRef(f"{dbpedia}birthDate")
deaddate = URIRef(f"{dbpedia}deathDate")
created = URIRef(f"http://purl.org/dc/terms/created")

### Utilitary functions to inject knowledge

In [ ]:
def addInstance(instance, concept, kg):
    kg.add(triple=(instance, RDF.type, concept))


def addLabel(entity, label, kg):
    kg.add(triple=(entity, RDFS.label, Literal(label, datatype=XSD.string)))


def linkInstances(sub, rel, obj, kg):
    kg.add(triple=(sub, rel, obj))


def addDataproperty(sub, rel, obj, kg):
    kg.add(triple=(sub, rel, obj))

### KG Creation

In [ ]:
## creation of the kg
kg = Graph()

#### Person

In [ ]:
persons = [
    "Wolfgang Amadeus Mozart",
    "Leopold Mozart",
    "Constance Weber",
    "Claudio Ebbado",
]


for p in persons:
    p_formated = "".join(p.split(" "))
    addInstance(instance=URIRef(f"{namespace}{p_formated}"), concept=FOAF.Person, kg=kg)
    addLabel(entity=URIRef(f"{namespace}{p_formated}"), label=p, kg=kg)
    if "Wolfgang" in p:
        addInstance(
            instance=URIRef(f"{namespace}{p_formated}"), concept=compositeur, kg=kg
        )

#### Locations

In [ ]:
towns = ["Londres", "Salzbourg", "Vienne"]
for t in towns:
    t_formated = "".join(t.split(" "))
    addInstance(instance=URIRef(f"{namespace}{t_formated}"), concept=town, kg=kg)
    addLabel(entity=URIRef(f"{namespace}{t_formated}"), label=t, kg=kg)

##country
addInstance(instance=URIRef(f"{namespace}Autriche"), concept=country, kg=kg)
addLabel(entity=URIRef(f"{namespace}Autriche"), label=("Autriche"), kg=kg)

#### Artwork

In [ ]:
artworks = [
    "La Flute Enchantee",
    "Jupiter",
    "Allegro Vivace",
    "Andante Cantabile",
    "Menuetto",
    "Molto Allegro",
]

for art in artworks:
    art_formated = "".join(art.split(" "))
    addInstance(instance=URIRef(f"{namespace}{art_formated}"), concept=artwork, kg=kg)
    addLabel(entity=URIRef(f"{namespace}{art_formated}"), label=art, kg=kg)

#### Related to Mozard

In [ ]:
mozard = URIRef(f"{namespace}WolfgangAmadeusMozart")
linkInstances(
    sub=mozard,
    rel=hasspouse,
    obj=URIRef(f"{namespace}ConstanceWeber"),
    kg=kg,
)

linkInstances(
    sub=mozard,
    rel=sonOf,
    obj=URIRef(f"{namespace}LeopoldMozart"),
    kg=kg,
)

#### Birth

In [ ]:
bn_bornlocation = BNode(value="#bornlocation")
linkInstances(sub=mozard, rel=isborn, obj=bn_bornlocation, kg=kg)

addDataproperty(
    sub=bn_bornlocation,
    rel=birthdate,
    obj=Literal("1756-01-27", datatype=XSD.date),
    kg=kg,
)

linkInstances(
    sub=bn_bornlocation,
    rel=birthplace,
    obj=URIRef(f"{namespace}Salzbourg"),
    kg=kg,
)

#### Death

In [ ]:
bn_deadlocation = BNode(value="#deadlocation")
linkInstances(sub=mozard, rel=isDead, obj=bn_deadlocation, kg=kg)

addDataproperty(
    sub=bn_deadlocation,
    rel=deaddate,
    obj=Literal("1791-12-27", datatype=XSD.date),
    kg=kg,
)

linkInstances(
    sub=bn_deadlocation,
    rel=deadplace,
    obj=URIRef(f"{namespace}Vienne"),
    kg=kg,
)


linkInstances(
    sub=URIRef(f"{namespace}Autriche"),
    rel=hascapital,
    obj=URIRef(f"{namespace}Vienne"),
    kg=kg,
)

#### Mozart ArtWork

In [ ]:
## Mozart ArtWork
linkInstances(
    sub=mozard,
    rel=created,
    obj=URIRef(f"{namespace}LaFluteEnchantee"),
    kg=kg,
)

linkInstances(
    sub=mozard,
    rel=created,
    obj=URIRef(f"{namespace}Jupiter"),
    kg=kg,
)


jupyerparts = [
    "Allegro Vivace",
    "Andante Cantabile",
    "Menuetto",
    "Molto Allegro",
]

for part in jupyerparts:
    part_formated = "".join(part.split(" "))
    linkInstances(
        sub=URIRef(f"{namespace}Jupiter"),
        rel=hasPart,
        obj=URIRef(f"{namespace}{part_formated}"),
        kg=kg,
    )


linkInstances(
    sub=URIRef(f"{namespace}Jupiter"),
    rel=direct_by,
    obj=URIRef(f"{namespace}ClaudioEbbado"),
    kg=kg,
)

## Saving

In [ ]:
kg.serialize(destination="myKG.ttl", format="turtle")
kg.serialize(destination="myKG.rdf", format="pretty-xml")
kg.serialize(destination="myKG.n3", format="n3")

<Graph identifier=Nc85f25a65d2341fe9abe0d23f78d8719 (<class 'rdflib.graph.Graph'>)>

### VIsualisation

In [ ]:
dico_typing = dict()
for subj, pred, obj in kg:
    if "type" in pred:
        dico_typing[str(subj)] = str(obj)

dico_typing

{'http://mygraph.com/AllegroVivace': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/LaFluteEnchantee': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/Londres': 'http://dbpedia.org/ontology/Town',
 'http://mygraph.com/Menuetto': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/AndanteCantabile': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/Jupiter': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/Salzbourg': 'http://dbpedia.org/ontology/Town',
 'http://mygraph.com/Autriche': 'http://dbpedia.org/ontology/Country',
 'http://mygraph.com/MoltoAllegro': 'http://dbpedia.org/ontology/Artwork',
 'http://mygraph.com/Vienne': 'http://dbpedia.org/ontology/Town'}

In [ ]:
instances_colors = {
    f"{namespace}Londres": "cyan",
    f"{namespace}Autriche": "yellow",
    f"{namespace}WolfgangAmadeusMozart": "red",
    f"{namespace}Menuetto": "pink",
    f"{namespace}AllegroVivace": "pink",
    f"{namespace}MoltoAllegro": "pink",
    f"{namespace}AndanteCantabile": "pink",
    f"{namespace}Salzbourg": "cyan",
    f"{namespace}LeopoldMozart": "green",
    f"{namespace}LaFluteEnchantee": "pink",
    f"{namespace}Vienne": "cyan",
    f"{namespace}ClaudioEbbado": "green",
    f"{namespace}ConstanceWeber": "green",
    f"{namespace}Jupiter": "pink",
    f"{dbpedia}Town": "cyan",
    f"{kzk}music#Composer": "#ff7f0e",
    f"{foaf}Person": "green",
    f"{dbpedia}Artwork": "pink",
    f"{dbpedia}Country": "purple",
}

In [ ]:
nodes_set = set()
edges_set = set()
for subj, pred, obj in kg:
    nodes_set.add(str(obj))
    nodes_set.add(str(subj))
    edges_set.add(tuple([str(subj), str(pred), str(obj)]))

In [ ]:
from pyvis.network import Network

In [ ]:
kgNet = Network(
    height="600px",
    width="100%",
    notebook=True,
    directed=True,
    neighborhood_highlight=True,
    # layout="forceAtlas2Based",
    cdn_resources="remote",
)

# kgNet.add_nodes(nodes_set)
for node in nodes_set:

    kgNet.add_node(
        n_id=node.replace(namespace, "ex:")
        .replace(dbpedia, "dbpedia:")
        .replace(foaf, "foaf:")
        .replace(schema, "schema:")
        .replace(kzk, "kzk"),
        color=instances_colors[node] if node in instances_colors.keys() else "#97c2fc",
    )

for edge in edges_set:
    kgNet.add_edge(
        source=edge[0]
        .replace(namespace, "ex:")
        .replace(dbpedia, "dbpedia:")
        .replace(foaf, "foaf:")
        .replace(schema, "schema:")
        .replace(kzk, "kzk"),
        to=edge[2]
        .replace(namespace, "ex:")
        .replace(dbpedia, "dbpedia:")
        .replace(foaf, "foaf:")
        .replace(schema, "schema:")
        .replace(kzk, "kzk"),
        label=edge[1].split("#")[-1].split("/")[-1],
    )

kgNet.show_buttons(filter_=["physics"])
kgNet.show("myKG.html")

myKG.html
